In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt



1. Moving Average Filter:
   - Mengurangi noise dengan cara merata-ratakan piksel di sekitar
   - Efektif untuk smoothing gambar sederhana
   - Dapat menghilangkan detail halus gambar

In [3]:

def moving_average_filter(image, kernel_size=5):
    """
    Menerapkan filter moving average untuk smoothing gambar

    Parameters:
    - image: Gambar input
    - kernel_size: Ukuran kernel untuk filter (default 5x5)

    Returns:
    - Gambar setelah difilter moving average
    """
    # Membuat kernel moving average
    kernel = np.ones((kernel_size, kernel_size), np.float32) / (kernel_size**2)

    # Aplikasikan filter konvolusi
    smoothed = cv2.filter2D(image, -1, kernel)

    return smoothed

2. SIFT Feature Detection:
   - Mendeteksi titik-titik kunci yang invariant terhadap skala dan rotasi
   - Berguna untuk pencocokan fitur antar gambar
   - Menghasilkan deskriptor unik untuk setiap keypoint

In [4]:
def sift_feature_detection(image):
    """
    Deteksi fitur menggunakan algoritma SIFT (Scale Invariant Feature Transform)

    Parameters:
    - image: Gambar input grayscale

    Returns:
    - Gambar dengan fitur SIFT yang terdeteksi
    """
    # Inisialisasi detector SIFT
    sift = cv2.SIFT_create()

    # Temukan keypoints dan deskriptor
    keypoints, descriptors = sift.detectAndCompute(image, None)

    # Gambar keypoints pada gambar
    image_with_keypoints = cv2.drawKeypoints(image, keypoints, None,
                                             flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

    return image_with_keypoints, keypoints, descriptors

3. Gaussian Smoothing:
   - Lebih canggih dibanding moving average
   - Memberikan bobot lebih pada piksel pusat
   - Mempertahankan lebih banyak detail gambar
   - Berguna untuk pre-processing sebelum deteksi fitur
   

In [5]:
def gaussian_smoothing(image, kernel_size=(5,5), sigma=0):
    """
    Melakukan Gaussian smoothing pada gambar

    Parameters:
    - image: Gambar input
    - kernel_size: Ukuran kernel Gaussian
    - sigma: Standar deviasi Gaussian (0 berarti dihitung otomatis)

    Returns:
    - Gambar setelah Gaussian smoothing
    """
    # Aplikasikan Gaussian blur
    smoothed = cv2.GaussianBlur(image, kernel_size, sigma)

    return smoothed

4. Sobel Edge Detection:
   - Mendeteksi perubahan intensitas tajam di gambar
   - Menghasilkan gradient horizontal dan vertikal
   - Membantu mengidentifikasi tepi dan struktur gambar

In [6]:
def sobel_edge_detection(image):
    """
    Deteksi tepi menggunakan operator Sobel

    Parameters:
    - image: Gambar input grayscale

    Returns:
    - Gradient x, gradient y, dan magnitude tepi
    """
    # Hitung gradien x dan y
    grad_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)

    # Hitung magnitude gradien
    magnitude = np.sqrt(grad_x**2 + grad_y**2)

    # Normalisasi untuk visualisasi
    magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    return grad_x, grad_y, magnitude


5. HOG Feature Descriptor:
   - Menangkap informasi gradien dan orientasi
   - Sangat efektif untuk deteksi objek
   - Umum digunakan dalam computer vision, terutama deteksi manusia

In [7]:
def hog_feature_descriptor(image, cell_size=(8,8), block_size=(2,2), nbins=9):
    """
    Ekstraksi fitur Histogram of Oriented Gradients (HOG)

    Parameters:
    - image: Gambar input
    - cell_size: Ukuran sel untuk HOG (dalam piksel)
    - block_size: Ukuran blok (dalam jumlah sel)
    - nbins: Jumlah bin histogram

    Returns:
    - Deskriptor HOG dan visualisasi
    """
    # Pastikan gambar dalam skala keabuan
    if len(image.shape) > 2:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image

    # Hitung HOG
    win_size = (image.shape[1] // cell_size[0] * cell_size[0],
                image.shape[0] // cell_size[1] * cell_size[1])

    # Block stride biasanya sama dengan cell size
    block_stride = (cell_size[0], cell_size[1])

    # Konversi block_size dari jumlah sel ke piksel
    block_size_pixels = (block_size[0] * cell_size[0],
                        block_size[1] * cell_size[1])

    hog = cv2.HOGDescriptor(win_size,
                           block_size_pixels,  # Ukuran blok dalam piksel
                           block_stride,
                           cell_size,
                           nbins)

    # Hitung descriptor
    hog_features = hog.compute(gray)

    return hog_features

In [8]:
def visualize_results(original, moving_avg, gaussian, sobel_mag, sift_img, hog_features):
    """
    Visualisasi hasil pemrosesan gambar

    Parameters:
    - Berbagai hasil pemrosesan gambar
    """
    plt.figure(figsize=(20,15))

    # Plot original
    plt.subplot(2,3,1)
    plt.title('Gambar Asli')
    plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
    plt.axis('off')

    # Plot Moving Average
    plt.subplot(2,3,2)
    plt.title('Moving Average Filter')
    plt.imshow(cv2.cvtColor(moving_avg, cv2.COLOR_BGR2RGB))
    plt.axis('off')

    # Plot Gaussian Smoothing
    plt.subplot(2,3,3)
    plt.title('Gaussian Smoothing')
    plt.imshow(cv2.cvtColor(gaussian, cv2.COLOR_BGR2RGB))
    plt.axis('off')

    # Plot Sobel Edge Detection
    plt.subplot(2,3,4)
    plt.title('Sobel Edge Detection')
    plt.imshow(sobel_mag, cmap='gray')
    plt.axis('off')

    # Plot SIFT Keypoints
    plt.subplot(2,3,5)
    plt.title('SIFT Feature Detection')
    plt.imshow(cv2.cvtColor(sift_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')

    # Plot HOG Histogram
    plt.subplot(2,3,6)
    plt.title('HOG Feature Descriptor')
    plt.bar(range(len(hog_features)), hog_features.flatten())
    plt.xlabel('HOG Feature Index')
    plt.ylabel('Feature Magnitude')

    plt.tight_layout()
    plt.show()


In [9]:
def main():
    # Baca gambar
    image = cv2.imread('/content/sample_data/Screenshot (1361).jpg')

    if image is None:
        print("Gagal membaca gambar. Pastikan file 'sample_image.jpg' tersedia.")
        return

    # Konversi ke grayscale untuk beberapa operasi
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Terapkan Moving Average Filter
    moving_avg_filtered = moving_average_filter(image)

    # Gaussian Smoothing
    gaussian_smoothed = gaussian_smoothing(image)

    # Deteksi Tepi Sobel
    grad_x, grad_y, sobel_magnitude = sobel_edge_detection(gray)

    # SIFT Feature Detection
    sift_img, keypoints, descriptors = sift_feature_detection(gray)

    # HOG Feature Descriptor
    hog_features = hog_feature_descriptor(image)

    # Visualisasi Hasil
    visualize_results(
        original=image,
        moving_avg=moving_avg_filtered,
        gaussian=gaussian_smoothed,
        sobel_mag=sobel_magnitude,
        sift_img=sift_img,
        hog_features=hog_features
    )

In [ ]:
if __name__ == "__main__":
    main()

permisi kak ini dari google collab-nya sepertinya ga kuat ngejalaninnya, soalnya compilenya lama banget, kemaren sampe 20 menit lebih abis itu crash sendiri, nanti di update kalau sudah bisa.

